# Writer Monad

## Outline

* Incentive for the Writer monad

* Definition of the Writer monad

* Writer monad examples

In this lesson, we will learn about the Writer monad type and how you can use it.

## Incentive for Writer monad

In the previous lecture we learned that with the Reader monad we can more easily pass data around by accessing the environment variable of the Reader monad. 

But what if we also need to modify this data. Let's look at an example where you process a string depending on it's length and count the operations peformed on the string.

In [ ]:
check1 :: String -> (Int, String)
check1 input = if length input < 5
  then check2 (1, input ++ "_below5")
  else check3 (1, input ++ "_above4")

check2 :: (Int, String) -> (Int, String)
check2 (operations, input) = if length input < 10
  then (operations, input)
  else (operations + 1, input ++ "_above9")

check3 :: (Int, String) -> (Int, String)
check3 (operations, input) = if length input < 16
  then check2 (operations + 1, removeFirst4 input)
  else check4 (operations + 1, input ++ "_above15")
  where removeFirst4 xs = [(!! id) xs | id <- [3..((length xs) - 1)]]

check4 :: (Int, String) -> (Int, String)
check4 (operations, input) = if length input < 30
  then (operations + 1 , input ++ "_above29")
  else (operations + 1, input ++ "_below30")

main :: IO ()
main = do
  let (count1, result1) = check1 "test"
  print "Processing word: test"
  print $ "Operations performed: " ++ show count1
  print $ "Final result: " ++ result1
  print "------------------------"
  let (count2, result2) = check1 "testing"
  print "Processing word: testing"
  print $ "Operations performed: " ++ show count2
  print $ "Final result: " ++ result2

main

Passing data around and modifying it can get tedious when programs get bigger. 

For this reason the **Writer monad** was created that allows you to pass data and update it between different Writer monads. 

## Definition of the Writer monad

The definition of the **Writer monad** is following: 
```haskell
type Writer w = WriterT w Identity
data WriterT w m a 
```
Helper function for **Writer** monad:
```haskell
runWriter :: Monoid w => Writer w a -> (a, w) 
```

Another definition shown below uses multi-parameter type classes and funDeps, which are not standard Haskell 2010. 
```haskell
newtype Writer w a = Writer { runWriter :: (a,w) }

instance (Monoid w) => Monad (Writer w) where
    return a             = Writer (a,mempty)
    (Writer (a,w)) >>= f = let (a',w') = runWriter $ f a in Writer (a',w `mappend` w')
```

To make use of the Writer monad we help ourself with the functions `tell` and `runWriter`.

The `tell` function appends data to the writter variable. The `runWriter` function can retrieve the final value of the Writer monad.

They are both included in the module `Control.Monad.Writer`. The `tell` function is defined in the `MonadWriter` type class.

```haskell
class (Monoid w, Monad m) => MonadWriter w m | m -> w where
    {-# MINIMAL (writer | tell), listen, pass #-}
    -- | @'writer' (a,w)@ embeds a simple writer action.
    writer :: (a,w) -> m a
    writer ~(a, w) = do
      tell w
      return a

    -- | @'tell' w@ is an action that produces the output @w@.
    tell   :: w -> m ()
    tell w = writer ((),w)

    -- | @'listen' m@ is an action that executes the action @m@ and adds
    -- its output to the value of the computation.
    listen :: m a -> m (a, w)
    -- | @'pass' m@ is an action that executes the action @m@, which
    -- returns a value and a function, and returns the value, applying
    -- the function to the output.
    pass   :: m (a, w -> w) -> m a

-- | @'listens' f m@ is an action that executes the action @m@ and adds
-- the result of applying @f@ to the output to the value of the computation.
--
-- * @'listens' f m = 'liftM' (id *** f) ('listen' m)@
listens :: MonadWriter w m => (w -> b) -> m a -> m (a, b)
listens f m = do
    ~(a, w) <- listen m
    return (a, f w)

-- | @'censor' f m@ is an action that executes the action @m@ and
-- applies the function @f@ to its output, leaving the return value
-- unchanged.
--
-- * @'censor' f m = 'pass' ('liftM' (\\x -> (x,f)) m)@
censor :: MonadWriter w m => (w -> w) -> m a -> m a
censor f m = pass $ do
    a <- m
    return (a, f)
```

Here is a simple example how to define a Writer monad and use the `tell` and `runWriter` functions.

In [ ]:
import Control.Monad.Writer

logAge :: Int -> Writer [String] Int  
logAge x = do
    tell ["Age is: " ++ show x]
    return x            
      
sumAge :: Writer [String] Int  
sumAge = do  
    a <- logAge 16  
    b <- logAge 18
    tell ["Summed age is: " ++ (show $ a + b)]
    return (a + b)

print $ runWriter sumAge

We see that every time we use the `tell` function the value gets appended to the first parameter of the Writer monad. 

The type of the parameter which is in our case a list has to have an instance of the **Monoid** type class, which lists have.

## Writer monad examples

Let's implement the example from the first chapter with the use of the Writer monad.

In [ ]:
instance Semigroup Int where
  a <> b = a + b

instance Monoid Int where
  mempty = 0

check1' :: String -> (Int, String)
check1' input = if length input < 5
  then swap $ runWriter $ do
    tell 1
    check2' $ input ++ "_below5"
  else swap $ runWriter $ do
    tell 1
    check3' $ input ++ "_above4"
  where swap (x,y) = (y,x)

check2' :: String -> Writer Int String
check2' input = if length input < 10
  then do
    return input
  else do
    tell 1
    return $ input ++ "_above9"

check3' :: String -> Writer Int String
check3' input = if length input < 16
  then do
    tell 1
    check2' $ removeFirst4 input
  else do
    tell 1
    check4' $ input ++ "_above15"
  where removeFirst4 xs = [(!! id) xs | id <- [3..((length xs) - 1)]]

check4' :: String -> Writer Int String
check4' input = if length input < 10
  then do
    tell 1
    return $ input ++ "_above29"
  else do
    tell 1
    return $ input ++ "_below30"

main2 :: IO ()
main2 = do
  let (count1, result1) = check1' "test"
  print "Processing word: test"
  print $ "Operations performed: " ++ show count1
  print $ "Final result: " ++ result1
  print "------------------------"
  let (count2, result2) = check1' "testing"
  print "Processing word: testing"
  print $ "Operations performed: " ++ show count2
  print $ "Final result: " ++ result2

main2

Since the type of the write parameter has to have an instance of Monoid we implement it in the begining for `Int`. 

After that we define the check functions similar as in the previous example where we use the `tell` function to keep track of the operations we have performed. 

Here is another example where we use also the `listen` function that enables us to retrieve the logs from the Writer monad.

In [ ]:
type Adding a = Writer [String] a

logMsg :: String -> Adding ()
logMsg msg = tell [msg]

add1 :: Int -> Adding Int
add1 x = do
    logMsg "Starting add1."
    let y = x + 1
    logMsg $ "Computed result: " ++ show y
    return y

start :: Int -> Adding Int
start x = do
    (n, logs) <- listen $ add1 x
    let logLines = length logs
    logMsg $ "add1 logged " ++ show logLines ++ " lines"
    return $ n

main :: IO ()
main = do
    print "Input an integer number:"
    n <- (read <$> getLine) :: IO Int
    let (result, logs) = runWriter $ start n
    print $ "Result is: " ++ show result
    putStrLn "Logs are: "
    mapM_ print logs

## Recap

In this lesson we've discussed:

- the motivation for introducing the Writer monad type 

- the definition of the Writer monad type and a simple example

- examples that use the Writer monad type to solve a problem